In [28]:
import requests
import psycopg2
import pandas as pd
import re


def compute_positions(start_time):
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5432')
    cursor = connection.cursor()
    query = f"""
        SELECT clearing_account, symbol, SUM(ABS(quantity)) AS position
        FROM trades
        WHERE trading_date = timestamp '{start_time}'
        AND clearing_account NOT IN ('5646353', '3618282', '3618588', '3616407', '5647492')
        AND clearing_account NOT LIKE 'AOS%'
        GROUP BY clearing_account, symbol
        limit 10
        """
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()
    connection.close()

    df = pd.DataFrame(rows, columns=['Trader', 'Underlying', 'Position'])
    return df

def remove_suffix(symbol):
    # Extract the first group of capital letters and remove trailing characters
    matches = re.match(r"([A-Z]+)", symbol)
    if matches:
        symbol = matches.group(1)
    return symbol

date = '2023-06-01'
positions = compute_positions(date)

# Example usage
symbols = positions['Underlying'].tolist()

url = "https://twelve-data1.p.rapidapi.com/time_series"

headers = {
    "X-RapidAPI-Key": "e902ad106dmsh3e8e389d32866d8p113ddejsnc1b66501d82a",
    "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
}

volumes = {}
for symbol in symbols:
    symbol = remove_suffix(symbol) 
    querystring = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": "1",
        "format": "json"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    print(f"Symbol: {symbol}, Response: {data}")
    
    if 'values' in data and len(data['values']) > 0:
        volume = data['values'][0]['volume']
        volumes[symbol] = volume
    else:
        print(f"No volume data found for symbol: {symbol}")

# Print the volumes for each symbol
for symbol, volume in volumes.items():
    print(f"Symbol: {symbol}, Volume: {volume}")

Symbol: MDB, Response: {'meta': {'symbol': 'MDB', 'interval': '1day', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'mic_code': 'XNMS', 'type': 'Common Stock'}, 'values': [{'datetime': '2023-06-02', 'open': '380.39001', 'high': '397.97989', 'low': '370.00000', 'close': '373.78320', 'volume': '7218753'}], 'status': 'ok'}
Symbol: LRCX, Response: {'meta': {'symbol': 'LRCX', 'interval': '1day', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'mic_code': 'XNGS', 'type': 'Common Stock'}, 'values': [{'datetime': '2023-06-02', 'open': '618.98499', 'high': '621.98999', 'low': '608.05389', 'close': '614.73999', 'volume': '618287'}], 'status': 'ok'}
Symbol: CRWD, Response: {'meta': {'symbol': 'CRWD', 'interval': '1day', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'mic_code': 'XNGS', 'type': 'Common Stock'}, 'values': [{'datetime': '2023-06-02', 'open': '158.53000', 'high': '159.39999', 'low

In [1]:
import psycopg2
from collections import defaultdict
from datetime import datetime
import requests
from datetime import datetime
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

def download_and_group_volume_data(chrome_driver_path, download_directory):
    csv_file_path = os.path.join(download_directory, "vol-query-results.csv")

    # Check if the file already exists
    if os.path.exists(csv_file_path):
        print("File already exists. Skipping download.")
    else:
        url = "https://www.theocc.com/market-data/market-data-reports/volume-and-open-interest/volume-query"

        chrome_options = Options()
        chrome_options.add_experimental_option(
            "prefs", {
                "download.default_directory": download_directory,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            }
        )

        driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)
        driver.get(url)

        # Click on the "Accept" button for the cookies banner
        accept_cookies_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.CookieConsent > span.ConsentButtons > a:nth-child(2)"))
        )
        accept_cookies_button.click()

        # Locate the download button using the XPath and click it
        download_button = WebDriverWait(driver, 10).until(
            lambda d: d.find_element(By.XPATH, '/html/body/div/main/section/div/div/div[2]/div/div/div[9]/div/a')
        )
        download_button.click()

        # Add some delay to let the download complete (adjust the time as needed)
        time.sleep(5)

        print(f"File downloaded to: {download_directory}")

        driver.quit()

    # Read the CSV file into a DataFrame
    volume_df = pd.read_csv(csv_file_path)

    # Group the data by 'underlying' and sum the 'quantity' column
    grouped_df = volume_df.groupby('underlying')['quantity'].sum().reset_index()

    return grouped_df

def get_grouped_df(chrome_driver_path, download_directory):
    grouped_df = download_and_group_volume_data(chrome_driver_path, download_directory)
    grouped_df = grouped_df.rename(columns={'underlying': 'Underlying'})
    grouped_df = grouped_df.rename(columns={'quantity': 'Volume'})

    return grouped_df

def compute_positions(start_time):
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5432')
    cursor = connection.cursor()
    query = f"""
        SELECT clearing_account, symbol, SUM(ABS(quantity)) AS position
        FROM trades
        WHERE trading_date = timestamp '{start_time}'
        AND clearing_account NOT IN ('5646353', '3618282', '3618588', '3616407', '5647492')
        AND clearing_account NOT LIKE 'AOS%'
        GROUP BY clearing_account, symbol
        """
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()
    connection.close()

    df = pd.DataFrame(rows, columns=['Trader', 'Underlying', 'Position'])
    return df

date = '2023-06-02'
chrome_driver_path = "/usr/bin/chromedriver"
download_directory = "/home/elliott/Development/scripts/jupyter_notebooks/"
grouped_df = get_grouped_df(chrome_driver_path, download_directory)
positions = compute_positions(date)
positions.to_csv("positions.csv")
merged_df = positions.merge(grouped_df, on='Underlying', how='inner')
merged_df['Position'] = merged_df['Position'].astype(float)
merged_df['Volume'] = merged_df['Volume'].astype(float)
merged_df['Concentration'] = (merged_df['Position'] / merged_df['Volume']) * 100
merged_df['Concentration'] = merged_df['Concentration'].round(2)
merged_df['Concentration'] = merged_df['Concentration'].map("{:.2f}%".format)
merged_df['Concentration'] = merged_df['Concentration'].str.rstrip('%').astype('float')
merged_df.sort_values(by='Concentration', axis=0, ascending=False, inplace=True)
merged_df.to_csv(f"concentration_report_{date}.csv")

/tmp/ipykernel_605707/1590350055.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


File downloaded to: /home/elliott/Development/scripts/jupyter_notebooks/


In [8]:
grouped_df

,Underlying,Volume
0,A,824
1,AA,77512
2,AAC,10
3,AADI,10
4,AAIC,234
...,...,...
4084,ZUO,234
4085,ZVRA,134
4086,ZWS,36
4087,ZYME,498


In [14]:
import requests

symbols = ['AAPL', 'GOOGL', 'MSFT']  # List of symbols

url = "https://twelve-data1.p.rapidapi.com/time_series"

headers = {
    "X-RapidAPI-Key": "e902ad106dmsh3e8e389d32866d8p113ddejsnc1b66501d82a",
    "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
}

volumes = {}
for symbol in symbols:
    querystring = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": "1",
        "format": "json"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    if 'values' in data and len(data['values']) > 0:
        volume = data['values'][0]['volume']
        volumes[symbol] = volume
    else:
        print(f"No volume data found for symbol: {symbol}")

# Print the volumes for each symbol
for symbol, volume in volumes.items():
    print(f"Symbol: {symbol}, Volume: {volume}")

Symbol: AAPL, Volume: 30522311
Symbol: GOOGL, Volume: 12942919
Symbol: MSFT, Volume: 11587144
